In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from recursive_RM import *
from itertools import product
import xarray as xr
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
# import stackprinter
# stackprinter.set_excepthook(style='lightbg')

In [11]:
def decode_func(llr, r, m, weight, decoder):
    # basic settings for all decoders
    kwargs = {'weight': weight, 'damp': 1, 'llr_clip': 30}
    # extra settings for some decoders
    if decoder.__name__ in ['RPA', 'CPA']:
        kwargs['base_dec'] = 'soft'
        kwargs['weight'] = {(3, 6): 1 / 651}
    return decoder(llr[inv_perm], r, m, **kwargs)[perm]


r, m, weight = 3, 6, {}
dB_range = np.array([2,3])
maxCw = 100

algor_list = ['RPA', 'CPA', 'RXA', 'CXA']

# initialize data structure to store results
K = len(algor_list)
InfBitErr = -np.ones([dB_range.size, maxCw, K], dtype=int)
CwBitErr = -np.ones([dB_range.size, maxCw, K], dtype=int)
CwErr = -np.ones([dB_range.size, maxCw, K], dtype=int)
rng = np.random.RandomState(1001)

Gsys, perm, inv_perm = systemize(RM(r, m))
for i, EbNo_dB in enumerate(dB_range):
    for t in tqdm(range(maxCw), total=maxCw):
        _, c, _, llr = AWGN_output(Gsys, EbNo_dB, rng)
        for j, algor in enumerate(algor_list):
            c_hat = decode_func(llr, r, m, weight, eval(algor))
            diff = abs(c - c_hat)
            # number of information bits errors
            InfBitErr[i, t, j] = diff[:Gsys.shape[0]].sum()
            # number of total bit errors in the entire codeword (b > a)
            CwBitErr[i, t, j] = diff.sum()
            # if the codeword is in error (1 or 0)
            CwErr[i, t, j] = diff.any()
        print(t, 'word_err', CwErr[i, :t+1].sum(axis=0), 'bit_err', CwBitErr[i, :t+1].sum(axis=0))

0 word_err [0 0 0 0] bit_err [0 0 0 0]
1 word_err [0 0 0 0] bit_err [0 0 0 0]
2 word_err [0 0 0 0] bit_err [0 0 0 0]
3 word_err [0 0 0 0] bit_err [0 0 0 0]
4 word_err [0 0 0 0] bit_err [0 0 0 0]
5 word_err [0 0 0 0] bit_err [0 0 0 0]
6 word_err [0 0 0 0] bit_err [0 0 0 0]
7 word_err [0 0 0 0] bit_err [0 0 0 0]
8 word_err [0 0 0 0] bit_err [0 0 0 0]
9 word_err [0 0 0 0] bit_err [0 0 0 0]
10 word_err [1 1 1 1] bit_err [12 12 12 12]
11 word_err [1 1 1 1] bit_err [12 12 12 12]
12 word_err [1 1 1 1] bit_err [12 12 12 12]
13 word_err [1 1 1 1] bit_err [12 12 12 12]
14 word_err [1 1 1 1] bit_err [12 12 12 12]
15 word_err [1 1 1 1] bit_err [12 12 12 12]
16 word_err [1 1 1 1] bit_err [12 12 12 12]
17 word_err [1 1 1 1] bit_err [12 12 12 12]
18 word_err [1 1 1 1] bit_err [12 12 12 12]
19 word_err [1 1 1 1] bit_err [12 12 12 12]
20 word_err [1 1 1 1] bit_err [12 12 12 12]
21 word_err [1 1 1 1] bit_err [12 12 12 12]
22 word_err [2 2 2 2] bit_err [20 20 20 20]
23 word_err [2 2 2 2] bit_err [20 20 2

0 word_err [0 0 0 0] bit_err [0 0 0 0]
1 word_err [0 0 0 0] bit_err [0 0 0 0]
2 word_err [0 0 0 0] bit_err [0 0 0 0]
3 word_err [0 0 0 0] bit_err [0 0 0 0]
4 word_err [0 0 0 0] bit_err [0 0 0 0]
5 word_err [0 0 0 0] bit_err [0 0 0 0]
6 word_err [0 0 0 0] bit_err [0 0 0 0]
7 word_err [0 0 0 0] bit_err [0 0 0 0]
8 word_err [0 0 0 0] bit_err [0 0 0 0]
9 word_err [0 0 0 0] bit_err [0 0 0 0]
10 word_err [0 0 0 0] bit_err [0 0 0 0]
11 word_err [0 0 0 0] bit_err [0 0 0 0]
12 word_err [0 0 0 0] bit_err [0 0 0 0]
13 word_err [0 0 0 0] bit_err [0 0 0 0]
14 word_err [0 0 0 0] bit_err [0 0 0 0]
15 word_err [0 0 0 0] bit_err [0 0 0 0]
16 word_err [0 0 0 0] bit_err [0 0 0 0]
17 word_err [0 0 0 0] bit_err [0 0 0 0]
18 word_err [0 0 0 0] bit_err [0 0 0 0]
19 word_err [0 0 0 0] bit_err [0 0 0 0]
20 word_err [0 0 0 0] bit_err [0 0 0 0]
21 word_err [0 0 0 0] bit_err [0 0 0 0]
22 word_err [0 0 0 0] bit_err [0 0 0 0]
23 word_err [0 0 0 0] bit_err [0 0 0 0]
24 word_err [0 0 0 0] bit_err [0 0 0 0]
25 word_er